*This notebook is based on an excerpt from the [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; the content is available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).*

*The text is released under the [CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode), and code is released under the [MIT license](https://opensource.org/licenses/MIT). If you find this content useful, please consider supporting the work by [buying the book](http://shop.oreilly.com/product/0636920034919.do)!*

# Data Indexing and Selection

In [Chapter 2](02.00-Introduction-to-NumPy.ipynb), we looked in detail at methods and tools to access, set, and modify values in NumPy arrays.  Review this chapter to understand the full extent of the selection methods available.

These included indexing (e.g., ``arr[2, 1]``), slicing (e.g., ``arr[:, 1:5]``), masking (e.g., ``arr[arr > 0]``), fancy indexing (e.g., ``arr[0, [1, 5]]``), and combinations thereof (e.g., ``arr[:, [1, 5]]``).
Here we'll look at similar means of accessing and modifying values in Pandas ``Series`` and ``DataFrame`` objects.
If you have used the NumPy patterns, the corresponding patterns in Pandas will feel very familiar, though there are a few quirks to be aware of.

We'll start with the more salient case of the `DataFrame` and move onto the simpler case of the `Series`.

In [ ]:
import pandas as pd
import numpy as np

## Data Selection in DataFrame

Recall that a ``DataFrame`` acts in many ways like a two-dimensional or structured array, and in other ways like a dictionary of ``Series`` structures sharing the same index.
These analogies can be helpful to keep in mind as we explore data selection within this structure.

### Accessing a single column

The first analogy we will consider is the ``DataFrame`` as a dictionary of related ``Series`` objects.
Let's return to our example of areas and populations of states:

In [ ]:
area = pd.Series({'California': 423967, 'Texas': 695662,
                  'New York': 141297, 'Florida': 170312,
                  'Illinois': 149995})
pop = pd.Series({'California': 38332521, 'Texas': 26448193,
                 'New York': 19651127, 'Florida': 19552860,
                 'Illinois': 12882135})
data = pd.DataFrame({'area':area, 'pop':pop})
data

The individual ``Series`` that make up the columns of the ``DataFrame`` can be accessed via dictionary-style indexing of the column name:

In [ ]:
# access by dictionary-style indexing
data['area']

Equivalently, we can use attribute-style access with column names that are strings:

In [ ]:
# access with attribute-style access
data.area

This attribute-style column access actually accesses the exact same object as the dictionary-style access:

In [ ]:
# Check to see if access is the same
data.area is data['area']

Though this is a useful shorthand, keep in mind that it does not work for all cases!
For example, if the column names are not strings, or if the column names conflict with methods of the ``DataFrame``, this attribute-style access is not possible.
For example, the ``DataFrame`` has a ``pop()`` method, so ``data.pop`` will point to this rather than the ``"pop"`` column:

In [ ]:
# Challenges of attribute-style access
data.pop is data['pop']

In particular, you should avoid the temptation to try column assignment via attribute (i.e., use ``data['pop'] = z`` rather than ``data.pop = z``).

Like with the ``Series`` objects discussed earlier, this dictionary-style syntax can also be used to modify the object, in this case adding a new column:

In [ ]:
data['density'] = data['pop'] / data['area']
data

This shows a preview of the straightforward syntax of element-by-element arithmetic between ``Series`` objects; we'll dig into this further in [Operating on Data in Pandas](03.03-Operations-in-Pandas.ipynb).

### Accessing as row, column: `loc` and `iloc`

As mentioned previously, we can also view the ``DataFrame`` as an enhanced two-dimensional array.
We can examine the raw underlying data array using the ``values`` attribute:

In [ ]:
# get underlying values as numpy array
data.values

For array-style indexing, we need another convention.
Here Pandas again uses the ``loc`` and ``iloc`` indexers.  **Recall that in our previous exploration of series, explicit indices can be numerical, convoluting whether you're selecting by implicit or explicit index.  These two methods can disambiguate the approach.**

### `iloc`
Think of `iloc` as a reference to the _integer_ location, or _implicit_ index, or by position.  Using the ``iloc`` indexer, we can index the underlying array and the ``DataFrame`` index and column labels are maintained in the result.  The indexer expects the row position followed by the column position.

In [ ]:
data

In [ ]:
# index the Dataframe by position, row 0, column 0
print(data.iloc[0,0])

#index the dataframe; find row 0, column1
data.iloc[0,1]

In [ ]:
# slice the dataframe to get the first 3 rows and the first 2 columns
data.iloc[:3, :2]

### `loc`

Similarly, using the ``loc`` indexer we can index the underlying data in an array-like style but using the _explicit index_ and column names:

In [ ]:
#Get the california row
data.loc['California']

**Question**.  We just said that these indexers require the format `[row, column]`.  However, it seems like we omitted that here.  What do you think is happening?

**Gotcha!** Look at how this function appears.  It looks _very_ similar to `data['California']`.  However, these behaviors are decidedly different; always remember that `.loc` will expect a _row index_ as the first parameter in the bracket, whereas dictionary-like key-value lookup will expect a column index.  

In [ ]:
# Try to use loc to just access the column
data.loc['pop']

In [ ]:
# Correctly use loc to access the column
data.loc[:, 'pop']

In [ ]:
# Can also slice
data.loc[:'Illinois', :'pop'] #explicit indices include the endpoint

### Try it yourself!!
Using the `data` dataframe above, try to do the following selections:
1. Select the `density` column using `iloc`, `loc`, and dictionary indexing.
2. Select only the Florida and Illinois rows with all columns.
3. Select the `area` and `pop` columns with all rows.
4. Select California, New York, and Illinois densities.
5. **Bonus**: Repeat #4, but reverse the order (e.g., dataframe returned should have order Illinois, New York, California)

**HINT**: When trying to select multiple indices, you can pass in a list of indices (e.g., `data.loc[['John', 'Raheim'], :]`)

In [ ]:
# Answer to 1
print(data['density'])
print(data.loc[:, 'density'])
print(data.iloc[:, 2])

In [ ]:
#Answer to 2
display(data.iloc[3:])
data.loc[['Florida', 'Illinois'], :]

In [ ]:
#Answer to 3
display(data.iloc[:, :2])
data.loc[:, ['area', 'pop']]

In [ ]:
#Answer to 4
display(data.iloc[[0,2,4], 2])
display(data.iloc[::2, 2])
data.loc['California'::2, 'density']

In [ ]:
#Answer to 5
data.loc['Illinois'::-2, 'density']

### Masking and fancy indexing

Many different data access patterns can be used within these indexers.
For example, in the ``loc`` indexer we can combine boolean masking and fancy indexing as in the following.  Boolean masking evaluates a certain condition on your dataset, and then returns the true/false value of this condition.  The returned dataset can then be used to filter out the `false` or `true` values.  Additionally fancy indexing can allow you to select multiple columns using a list.

In [ ]:
# Look at the boolean masking result of data.density > 100
data.density > 100
#data.loc[data.density > 100]

In [ ]:
# Only look at selected columns
data.loc[data.density > 100, ['pop', 'density']]

In [ ]:
# Look at all rows that aren't Florida
data.loc[data.index != 'Florida']

In [ ]:
# Find all the columns that aren't pop
data.columns != 'pop'
data.loc[:, data.columns != 'pop']

Any of these indexing conventions may also be used to set or modify values; this is done in the standard way that you might be accustomed to from working with NumPy:

In [ ]:
# You can also use these selections to set values and not just access them
data.iloc[0, 2] = 91
data

To build up your fluency in Pandas data manipulation, I suggest spending some time with a simple ``DataFrame`` and exploring the types of indexing, slicing, masking, and fancy indexing that are allowed by these various indexing approaches.

### Try it yourself!!
Using the dataframe `work_data` below, answer the following questions:
1. Which states have densities > 90?  _How many_ states have densities > 90?
2. What are the populations of the states which have densities > 90?  (Find the states that have densities greater than 90 and then return only the population column)
3. Which states have areas between 150,000 and 700,000?  What are their corresponding densities?  **Hints**:  (1) `&` is the `and` operator compatible with Pandas Series.  (2) Remember operator precedence and how parentheses can help with that!
4. **Bonus**: Which states have populations within 5 million of the median population?  **Hint**: use `np.median`

In [ ]:
# Make sure to execute this cell
work_data = data.copy()

In [ ]:
#Answer to question 1
res = work_data.loc[work_data.loc[:, 'density'] > 90]
display(res)
print('The total number of states with density > 90 is: {0}'.format(res.shape[0]))

In [ ]:
#Answer to question 2
work_data.loc[work_data.loc[:, 'density']>90, 'pop']

In [ ]:
# Answer to question 3
work_data.loc[(work_data.loc[:, 'area'] > 150000) & (work_data.loc[:, 'area'] < 700000)]

In [ ]:
# Answer to question 4
med_pop = np.median(work_data['pop'])
display(work_data.loc[(work_data['pop']<med_pop + 5e6) & (work_data['pop']>med_pop - 5e6)])
work_data.loc[work_data['pop'].between(med_pop-5e6, med_pop+5e6)]

## On Pandas: Query
As we've already seen in previous sections, the power of the PyData stack is built upon the ability of NumPy and Pandas to push basic operations into C via an intuitive syntax: examples are vectorized/broadcasted operations in NumPy, and grouping-type operations in Pandas.
While these abstractions are efficient and effective for many common use cases, they often rely on the creation of temporary intermediate objects, which can cause undue overhead in computational time and memory use.

As of version 0.13 (released January 2014), Pandas includes some experimental tools that allow you to directly access C-speed operations without costly allocation of intermediate arrays.
These are the ``eval()`` and ``query()`` functions, which rely on the [Numexpr](https://github.com/pydata/numexpr) package.
In this notebook we will walk through their use and give some rules-of-thumb about when you might think about using them.

You can rewrite many of the above methods using strings in combination with query if they're operating on the columns of the dataframe.  This helps with speed and performance since temporary intermediate variables are not created, and also helps with readability.  Let's see how this would look below.

In [ ]:
# Answer to 1
work_data.loc[work_data.loc[:, 'density'] > 90]

## with query
work_data.query('density > 90')

In [ ]:
# Answer to 2
work_data.loc[work_data.loc[:, 'density']>90, 'pop']

#with query
work_data.query('density > 90')['pop']

In [ ]:
# Answer to 3
work_data.loc[(work_data.loc[:, 'area'] > 150000) & (work_data.loc[:, 'area'] < 700000)]

#with query
work_data.query('area > 150000 & area < 700000')

In [ ]:
# Answer to 4: use @ to reference variables that are not in the dataframe
med_pop = np.median(work_data['pop'])

#with query
work_data.query('pop > @med_pop - 5e6 & pop < @med_pop + 5e6')

### Additional indexing conventions

There are a couple extra indexing conventions that might seem at odds with the preceding discussion, but nevertheless can be very useful in practice.
First, while *indexing* refers to columns, *slicing* refers to rows:

In [ ]:
# Dictionary-like access allows you to slice on the rows
data['Florida':'Illinois']

Such slices can also refer to rows by number rather than by index:

In [ ]:
# Dictionary-like access also allows you to slice by implicit index
data[1:3]

Similarly, direct masking operations are also interpreted row-wise rather than column-wise:

In [ ]:
# Masking operations are implemented row wise
data[data.density > 100]

These two conventions are syntactically similar to those on a NumPy array, and while these may not precisely fit the mold of the Pandas conventions, they are nevertheless quite useful in practice.

## Data Selection in Series

As we saw in the previous section, a ``Series`` object acts in many ways like a one-dimensional NumPy array, and in many ways like a standard Python dictionary.
If we keep these two overlapping analogies in mind, it will help us to understand the patterns of data indexing and selection in these arrays.

### Series as dictionary

Like a dictionary, the ``Series`` object provides a mapping from a collection of keys to a collection of values:

In [ ]:
#pandas series for testing
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

In [ ]:
#access a single row
data['b']

We can also use dictionary-like Python expressions and methods to examine the keys/indices and values:

In [ ]:
# Use python expression with Series
'a' in data

In [ ]:
# Access the keys in a dictionary-like way
data.keys()

In [ ]:
# Access the items as sets of tuples
list(data.items())

``Series`` objects can even be modified with a dictionary-like syntax.
Just as you can extend a dictionary by assigning to a new key, you can extend a ``Series`` by assigning to a new index value:

In [ ]:
# Set values by dictionary-like syntax
data['e'] = 1.25
data

This easy mutability of the objects is a convenient feature: under the hood, Pandas is making decisions about memory layout and data copying that might need to take place; the user generally does not need to worry about these issues.

### Series as one-dimensional array

A ``Series`` builds on this dictionary-like interface and provides array-style item selection via the same basic mechanisms as NumPy arrays – that is, *slices*, *masking*, and *fancy indexing*.
Examples of these are as follows:

In [ ]:
# slicing by explicit index
data['a':'c']

In [ ]:
# slicing by implicit integer index
data[0:2]

In [ ]:
# masking
data[(data > 0.3) & (data < 0.8)]

In [ ]:
# fancy indexing
data[['a', 'e']]

Among these, slicing may be the source of the most confusion.
Notice that when slicing with an explicit index (i.e., ``data['a':'c']``), the final index is *included* in the slice, while when slicing with an implicit index (i.e., ``data[0:2]``), the final index is *excluded* from the slice.

### Indexers: loc, iloc, and ix

These slicing and indexing conventions can be a source of confusion.
For example, if your ``Series`` has an explicit integer index, an indexing operation such as ``data[1]`` will use the explicit indices, while a slicing operation like ``data[1:3]`` will use the implicit Python-style index.

In [ ]:
data = pd.Series(['a', 'b', 'c'], index=[1, 3, 5])
data

In [ ]:
# explicit index when indexing
data[1]

In [ ]:
# implicit index when slicing
data[1:3]

Because of this potential confusion in the case of integer indexes, Pandas provides some special *indexer* attributes that explicitly expose certain indexing schemes.
These are not functional methods, but attributes that expose a particular slicing interface to the data in the ``Series``.

First, the ``loc`` attribute allows indexing and slicing that always references the explicit index:

In [ ]:
# access using explicit index
data.loc[1]

In [ ]:
# Slice using explicit index
data.loc[1:3]

The ``iloc`` attribute allows indexing and slicing that always references the implicit Python-style index:

In [ ]:
# Accss using the implicit index
data.iloc[1]

In [ ]:
# Slice using the implicit index
data.iloc[1:3]

One guiding principle of Python code is that "explicit is better than implicit."
The explicit nature of ``loc`` and ``iloc`` make them very useful in maintaining clean and readable code; especially in the case of integer indexes, I recommend using these both to make code easier to read and understand, and to prevent subtle bugs due to the mixed indexing/slicing convention.

## What we've learned in this lesson:

1. Indexing and selection of `DataFrames` using:
    - Bracket indexing
    - `.iloc`
    - `.loc`
    - `.query`
2. Quirks of dataframe indexing vs slicing
3. Indexing and selection of `Series` using:
    - Bracket indexing
    - `.iloc`
    - `.loc`